In [ ]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import ImageFile
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (
    Dense,
    GlobalAveragePooling2D,
    Dropout,
    MaxPooling2D,
    Conv2D,
    Activation,
    concatenate,
)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from keras.layers import Layer
import keras.backend as K

np.random.seed(1)
tf.random.set_seed(1)

ImageFile.LOAD_TRUNCATED_IMAGES = True
results_folder = "path_to_your_Save_Results/"

In [ ]:
folders = glob.glob(r"Train_and_Vaid_Dataset_path\*")

img_list = []
label_list=[]

for folder in folders:
    print(folder) 
    for img in glob.glob(folder+r"/*.jpg"):
        n= cv2.imread(img)
        class_num = folders.index(folder)
        label_list.append(class_num)
        resized = cv2.resize(n, (299,299), interpolation = cv2.INTER_AREA)
        img_list.append(resized)
x_train, x_valid , y_train, y_valid  = train_test_split(img_list, label_list, test_size=0.2)
x_train = np.array(x_train)
y_train = np.array(y_train)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)
print ("training_set", x_train.shape)
print ("training_set", y_train.shape)
print ("validation_set",x_valid.shape)
print ("validation_set",y_valid.shape)
print("Train_Folder",len(x_train))
print("validation_Folder",len(x_valid))

In [ ]:
folders1 = glob.glob(r"Test_Dataset_path\*")

img_list1 = []
label_list1=[]

for folder1 in folders1:
    print(folder1) 
    for img1 in glob.glob(folder1+r"/*.jpg"):
        n1= cv2.imread(img1)
        class_num1 = folders1.index(folder1)
        label_list1.append(class_num1)
        resized1 = cv2.resize(n1, (299,299), interpolation = cv2.INTER_AREA)
        img_list1.append(resized1)

x_test, y_test = img_list1, label_list1
x_test = np.array(x_test)
y_test = np.array(y_test)
print ("test_set",x_test.shape)
print ("test_set",y_test.shape)
print("Test_Folder",len(x_test))

In [ ]:
class SoftAttention(Layer):
    def __init__(self, ch, m, concat_with_x=False, aggregate=False, **kwargs):
        self.channels = int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x
        super(SoftAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.i_shape = input_shape
        kernel_shape_conv3d = (self.channels, 3, 3) + (1, self.multiheads) # DHWC
        self.out_attention_maps_shape = input_shape[0:1] + (self.multiheads,) + input_shape[1:-1]
        if self.aggregate_channels == False:
            self.out_features_shape = input_shape[:-1] + (input_shape[-1] + (input_shape[-1] * self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1] + (input_shape[-1] * 2,)
            else:
                self.out_features_shape = input_shape

        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                              initializer='he_uniform',
                                              name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                            initializer='zeros',
                                            name='bias_conv3d')

        super(SoftAttention, self).build(input_shape)

    def call(self, x):
        exp_x = K.expand_dims(x, axis=-1)
        c3d = K.conv3d(exp_x,
                       kernel=self.kernel_conv3d,
                       strides=(1, 1, self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = K.bias_add(c3d, self.bias_conv3d)
        conv3d = kl.Activation('selu')(conv3d)  # Using SELU activation here

        conv3d = K.permute_dimensions(conv3d, pattern=(0, 4, 1, 2, 3))
        conv3d = K.squeeze(conv3d, axis=-1)
        conv3d = K.reshape(conv3d, shape=(-1, self.multiheads, self.i_shape[1] * self.i_shape[2]))
        softmax_alpha = K.softmax(conv3d, axis=-1)
        softmax_alpha = kl.Reshape(target_shape=(self.multiheads, self.i_shape[1], self.i_shape[2]))(softmax_alpha)

        if self.aggregate_channels == False:
            exp_softmax_alpha = K.expand_dims(softmax_alpha, axis=-1)
            exp_softmax_alpha = K.permute_dimensions(exp_softmax_alpha, pattern=(0, 2, 3, 1, 4))
            x_exp = K.expand_dims(x, axis=-2)
            u = kl.Multiply()([exp_softmax_alpha, x_exp])
            u = kl.Reshape(target_shape=(self.i_shape[1], self.i_shape[2], u.shape[-1] * u.shape[-2]))(u)
        else:
            exp_softmax_alpha = K.permute_dimensions(softmax_alpha, pattern=(0, 2, 3, 1))
            exp_softmax_alpha = K.sum(exp_softmax_alpha, axis=-1)
            exp_softmax_alpha = K.expand_dims(exp_softmax_alpha, axis=-1)
            u = kl.Multiply()([exp_softmax_alpha, x])

        if self.concat_input_with_scaled:
            o = kl.Concatenate(axis=-1)([u, x])
        else:
            o = u

        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape):
        return [self.out_features_shape, self.out_attention_maps_shape]

    def get_config(self):
        return super(SoftAttention, self).get_config()


In [ ]:
models=[MobileNetV3Large]
header=["MobileNetV3Large"]
for i in range(len(models)):
    net = models[i](weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    conv = net.layers[-1].output
    attention_layer, map2 = SoftAttention(aggregate=True, m=16, concat_with_x=False, ch=int(conv.shape[-1]), name='soft_attention')(conv)
    attention_layer = MaxPooling2D(pool_size=(2, 2), padding="same")(attention_layer)
    conv = MaxPooling2D(pool_size=(2, 2), padding="same")(conv)

    conv = concatenate([conv, attention_layer])
    conv = Activation('PReLU')(conv)
    conv = Dropout(0.2)(conv)

    output = GlobalAveragePooling2D()(conv)
    output = Dense(2, activation='softmax')(output)
    model = Model(inputs=net.input, outputs=output)

    batch_size = 8
    epochs = 2
    opt = SGD(lr=0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   
    weights_filepath = results_folder+'best_weights_{}.h5'.format(header[i])  

   
    model_checkpoint_callback = ModelCheckpoint(
        filepath=weights_filepath,
        save_best_only=True,
        monitor='val_accuracy',  
        mode='max', 
        save_weights_only=True,  
        verbose=1
    )

    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid, y_valid), callbacks=[model_checkpoint_callback])


In [ ]:
y_pred=model.predict(x_test)
y_pred=np.argmax(y_pred, axis=1)

target_names = ["Fire","Normal"] 
classification_report(y_test,y_pred,  target_names=target_names,output_dict=True)
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)

print('Classification Report')
print(classification_report(y_test, y_pred, target_names=target_names))

    
